In [1]:
%%HTML
<style>
    body {
        --vscode-font-family: "ComicSansMS"
    }
</style>

In [2]:
import pandas as pd
import numpy as np

## Testing out LDO_functions.py

In [3]:
## Calling the function from LDO_functions.py
from LDO_functions import estimate_amplifier_stages, estimate_GBW_stages
# Test the functions using available variables
csv_file = "sf_nmos_char_vg_1_3.csv"
PSRR_target_dB = -40
f_bw = 1e6
I_load = 10e-3
PM_target = 60
C_load = 11.4e-12
kgm_max = 25
I_budget = (5/100) * I_load  # 5% of I_load

In [4]:
GBW_stage1, GBW_stage2 = estimate_GBW_stages(PSRR_target_dB, f_bw, PM_target, C_load)

print(f"Estimated GBW_stage1: {GBW_stage1*1e-9:.2f} GHz")
print(f"Estimated GBW_stage2: {GBW_stage2*1e-9:.2f} GHz")

Estimated GBW_stage1: 0.92 GHz
Estimated GBW_stage2: 0.35 GHz


In [7]:
# Estimate amplifier stages using the current test parameters
result_test = estimate_amplifier_stages(csv_file, GBW_stage1, GBW_stage2, C_load, kgm_target=25)
print("Amplifier stages estimation (test):")
for k, v in result_test.items():
    if isinstance(v, float) or isinstance(v, np.floating):
        print(f"{k}: {v:.4e}")
    else:
        print(f"{k}: {v}")

Calculating I_req for stage 1 with C_out 947.78 pF, GBW_stage1 0.92 GHz, kgm_max 25
Required gm of the 1st stage of the amplifier for GBW_stage1 0.92 GHz and C_in 947.78 pF: 5501.09 mS
Required current of the 1st stage of the amplifier for kgm_max 25 : 220.04 mA
Stage 2 required current (I_req_stage2): 0.9925 mA
Stage 2 input capacitance (C_in_stage2): 947.78 pF
Stage 1 required current (I_req_stage1): 220.0437 mA
Total required current (I_req_total): 221.0362 mA
Amplifier stages estimation (test):
I_req_stage2: 9.9251e-04
C_in_stage2: 9.4778e-10
I_req_stage1: 2.2004e-01
I_req_total: 2.2104e-01
details_stage2: {'idx_kgm': 9, 'kgm': np.float64(25.00386507127838), 'kcsg_n': np.float64(3.514438674718297e-07), 'kcgd_n': np.float64(3.017461437348191e-07), 'cgs': np.float64(3.4881190860395424e-10), 'cgd': np.float64(2.994863705182221e-10), 'C_in': np.float64(9.477846496403986e-10)}


In [ ]:
def estimate_gbw_and_amplifier_stages(PSRR_target_dB, f_bw, PM_target, C_load, csv_file, kgm_target=25):
    """
    Combines GBW estimation and amplifier stage estimation.

    Returns:
        result: dict containing GBW_stage1, GBW_stage2, and amplifier stage estimation results.
    """
    GBW_stage1, GBW_stage2 = estimate_GBW_stages(PSRR_target_dB, f_bw, PM_target, C_load)
    amp_result = estimate_amplifier_stages(csv_file, GBW_stage1, GBW_stage2, C_load, kgm_target=kgm_target)
    result = {
        "GBW_stage1": GBW_stage1,
        "GBW_stage2": GBW_stage2,
        "amplifier_stages": amp_result
    }
    return result

In [8]:
## Calling the function from LDO_functions.py
from LDO_functions import estimate_amplifier_stages, estimate_GBW_stages, estimate_gbw_and_amplifier_stages
# Test the functions using available variables
csv_file = "sf_nmos_char_vg_1_3.csv"
PSRR_target_dB = -40
f_bw = 1e6
I_load = 10e-3
PM_target = 60
C_load = 11.4e-12
kgm_max = 25
I_budget = (5/100) * I_load  # 5% of I_load

ImportError: cannot import name 'estimate_gbw_and_amplifier_stages' from 'LDO_functions' (c:\Users\sakth\OneDrive\Documents\C_ID_repo\LDO\LDO_functions.py)